## Toy Example

In [1]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.cluster import AgglomerativeClustering

from sklearn.cluster import DBSCAN

import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from keras.layers import Input, Dense, merge,Dropout
from keras.models import Model
from tensorflow.python.client import device_lib
import keras
from keras import optimizers
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [2]:
import tensorflow as tf
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [44]:
size = 10000
X = np.random.randn(4,size)
X2 = np.random.randn(4,size)

In [45]:
X4 = [1]*int(size/3) + [3]*int(size/3) + [5]*int(size/3) + [1.5]


Primeiro definimos as variáveis utilizadas:

X0, X1, X2, X3 variáveis aleatórias geradas por uma distribuição normal

X4 Variável em que 1/3 os valores são -1, 1/3 são 0 e 1/3 são 1


Depois definimos a função para o Y dada por:

Y = X0 * X1 + X2 * X3 + X1 * X3 + X4 * X1

In [46]:
Y = X[0]*X[1]+X[2]*X[3]+X[1]*X[3] + X4*((X[1])-X[2])

In [47]:

Y2 = X2[0]+X2[1]+X2[2]+X2[3]+X2[0]*X2[1] + X4*((X2[1])-X2[2])

In [48]:
X = pd.DataFrame(np.transpose(X),columns= ['X0','X1','X2', 'X3'])
X2 = pd.DataFrame(np.transpose(X2),columns= ['X0','X1','X2', 'X3'])
X['X4'] = X4
X2['X4'] = X4

Dividimos os dados em treino e teste.

In [49]:
X_train, X_test, y_train, y_test = train_test_split(
     X, Y, test_size=0.33, random_state=42)

In [50]:
x4_train = X_train['X4']
X_train.drop('X4',axis=1,inplace=True)
x4_test = X_test['X4']
X_test.drop('X4',axis=1,inplace=True)

/home/guilherme/anaconda3/envs/tf36/lib/python3.6/site-packages/pandas/core/frame.py:4170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [51]:
X2_train, X2_test, y2_train, y2_test = train_test_split(
     X2, Y2, test_size=0.33, random_state=42)

In [52]:
ys_train = pd.DataFrame(np.transpose([y_train, y2_train]), columns = ['y1','y2'])
ys_test =  pd.DataFrame(np.transpose([y_test, y2_test]), columns = ['y1','y2'])

In [53]:
x42_train = X2_train['X4']
X2_train.drop('X4',axis=1,inplace=True)
x42_test = X2_test['X4']
X2_test.drop('X4',axis=1,inplace=True)

Treinamos um modelo de rede neural

In [56]:
model = MLPRegressor(hidden_layer_sizes=(50,50,50,50,50),activation='relu',max_iter=1000, verbose=1,n_iter_no_change = 30)

In [ ]:
model.fit(X_train,y_train)

Iteration 1, loss = 10.83718857
Iteration 2, loss = 3.86792843
Iteration 3, loss = 2.86249210
Iteration 4, loss = 2.77087292
Iteration 5, loss = 2.76781721
Iteration 6, loss = 2.74991989
Iteration 7, loss = 2.70765249
Iteration 8, loss = 2.68569266


Obtendo o R² de treino:

In [15]:
model.score(X_train,y_train)

0.7957101379657049

E o R² de teste:

In [16]:
model.score(X_test,y_test)

0.7941396080681464

In [17]:
def print_simple_pred_mae(X_train, value):
    x_a = X_train.copy()#drop(['error','y'],axis=1).copy()
    x_a['X1'] = x_a['X1'] + value
    c1 = x_a['X0']*x_a['X1']+x_a['X2']*x_a['X3']+x_a['X1']*x_a['X3'] + x4_train*((x_a['X1'])-x_a['X2'])
    cp = model.predict(x_a)
    print(mean_absolute_error(cp, c1))

In [18]:
print_simple_pred_mae(X_train, 0.3)
print_simple_pred_mae(X_train, -0.3)

1.6002515444544425
1.597771418378902


In [19]:
y_pred = model.predict(X_test)

y_train_pred = model.predict(X_train)

In [20]:
print(mean_absolute_error(y_train_pred,y_train))
print(mean_absolute_error(y_pred,y_test))

1.5618165832028463
1.563546673789024


In [21]:
r2_score(y_pred, y_test)

0.74709043645146

Calculamos o erro de predição.

In [22]:
model_error =   y_pred -y_test
train_error = y_train - y_train_pred

In [23]:
X_train['error'] = train_error
X_test['error'] = model_error
X_train['y'] = y_train
X_test['y'] = y_test

/home/guilherme/anaconda3/envs/tf36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/guilherme/anaconda3/envs/tf36/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/guilherme/anaconda3/envs/tf36/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

In [25]:
model2 = MLPRegressor(hidden_layer_sizes=(50,50,50,50,50),activation='relu',max_iter=1000,verbose=1,n_iter_no_change = 30,batch_size = 128)

In [26]:
model2.fit(X2_train,y2_train)

Iteration 1, loss = 3.31125027
Iteration 2, loss = 2.75450957
Iteration 3, loss = 2.73692985
Iteration 4, loss = 2.73073599
Iteration 5, loss = 2.72763062
Iteration 6, loss = 2.71821627
Iteration 7, loss = 2.72731451
Iteration 8, loss = 2.72244366
Iteration 9, loss = 2.71646672
Iteration 10, loss = 2.71681939
Iteration 11, loss = 2.70499164
Iteration 12, loss = 2.70190209
Iteration 13, loss = 2.70338777
Iteration 14, loss = 2.70818512
Iteration 15, loss = 2.70514729
Iteration 16, loss = 2.70373449
Iteration 17, loss = 2.70267264


/home/guilherme/anaconda3/envs/tf36/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPRegressor(activation='relu', alpha=0.0001, batch_size=128, beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(50, 50, 50, 50, 50), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=1000,
             momentum=0.9, n_iter_no_change=30, nesterovs_momentum=True,
             power_t=0.5, random_state=None, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=1, warm_start=False)

Obtendo o R² de treino:

In [27]:
model2.score(X2_train,y2_train)

0.8106932987479856

E o R² de teste:

In [28]:
model2.score(X2_test,y2_test)

0.8112253847985506

In [29]:
y2_test

array([ 1.07678687, 16.16609737, -1.49484458, ..., -7.92560256,
        2.97335431, -7.12707131])

X2_train['X1'] = X2_train['X1'] + 1
c2 = X2_train['X0']+X2_train['X1']+X2_train['X2']+X2_train['X3']+X2_train['X0']*X2_train['X1'] + x4_train*((X2_train['X1'])-X2_train['X2'])
cp2 = model.predict(X2_train)
mean_absolute_error(cp2, c2)

In [30]:
y2_pred = model.predict(X2_test)
y2_train_pred = model.predict(X2_train)

In [31]:
print(mean_absolute_error(y2_train_pred,y2_train))
print(mean_absolute_error(y2_pred,y2_test))

2.5847652578333733
2.5804741893795393


Calculamos o erro de predição.

In [ ]:
def print_simple_pred_mae2(X2_train, value):
    x2_a = X2_train.copy()#.drop(['error','y'],axis=1).copy()
    x2_a['X1'] = x2_a['X1'] + value
    c2 = x2_a['X0']+x2_a['X1']+x2_a['X2']+x2_a['X3']+x2_a['X0']*x2_a['X1'] + x4_train*((x2_a['X1'])-x2_a['X2'])
    c2p = model2.predict(x2_a)
    print(mean_absolute_error(c2p, c2))

In [ ]:
print_simple_pred_mae2(X2_train, 0.3)
print_simple_pred_mae2(X2_train, -0.3)

In [ ]:
model2_error =   y2_pred -y2_test
train2_error = y2_train - y2_train_pred

In [ ]:
X2_train['error'] = train2_error
X2_test['error'] = model2_error
X2_train['y'] = y2_train
X2_test['y'] = y2_test

In [ ]:
df_encoder_train = X_train.merge(X2_train,left_index=True,right_index=True)
df_encoder_test = X_test.merge(X2_test,left_index=True,right_index=True)

In [ ]:

df_outs_train = df_encoder_train[['y_y','y_x']]
df_outs_test = df_encoder_test[['y_y','y_x']]
df_features_train = df_encoder_train.drop(['error_x','error_y','y_y','y_x'], axis = 1)
df_features_test = df_encoder_test.drop(['error_x','error_y','y_y','y_x'], axis = 1)


In [ ]:
encoding_dim = 1
ncol = 12


In [ ]:
keras.backend.clear_session()

input_dim = Input(shape = (ncol, ))
input_dim2 = Input(shape = (ncol - 4, ))
# Encoder Layers
encoded1 = Dense(10, activation = 'relu')(input_dim)
encoded2 = Dense(8, activation = 'relu')(encoded1)
encoded3 = Dense(5, activation = 'relu')(encoded2)
encoded4 = Dense(3, activation = 'relu')(encoded3)
encoded5 = Dense(1, activation = 'relu')(encoded4)
encoded13 = Dense(encoding_dim, activation = 'linear')(encoded5)
merged = keras.layers.concatenate([encoded13, input_dim2], axis=-1)
# Decoder Layers
decoded1 = Dense(30, activation = 'tanh')(merged)
drop1 = Dropout(0.2)(decoded1)
decoded2 = Dense(40, activation = 'tanh')(drop1)
decoded3 = Dense(50, activation = 'tanh')(decoded2)
decoded4 = Dense(40, activation = 'tanh')(decoded3)
decoded5 = Dense(30, activation = 'tanh')(decoded4)
decoded13 = Dense(2, activation = 'linear')(decoded5)

# Combine Encoder and Deocder layers
autoencoder = Model(inputs = [input_dim, input_dim2], outputs = [decoded13])
sgd = optimizers.SGD(lr=0.1, decay=0.001, momentum=0.9, nesterov=True)

# Compile the Model
autoencoder.compile(optimizer = 'adam', loss = 'mean_absolute_error')

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='loss',
                              min_delta=0,
                              patience=800,
                              verbose=0, mode='auto',
                                  restore_best_weights=False)


callbacks = [es]
autoencoder.fit(x = [df_encoder_train, df_features_train], y = df_outs_train, epochs = 1000, 
                batch_size = 128, shuffle = True,
                
               callbacks = callbacks
                
               )

In [ ]:
encoder = Model(inputs = [input_dim,input_dim2], outputs = decoded13)
encoded_input = Input(shape = (encoding_dim, ))
encoded_train = pd.DataFrame(encoder.predict([df_encoder_train,df_features_train]))
encoded_train = encoded_train.add_prefix('feature_')

In [ ]:
print(mean_absolute_error(encoded_train['feature_0'], df_outs_train['y_y']))
print(mean_absolute_error(encoded_train['feature_1'], df_outs_train['y_x']))

In [ ]:
print(r2_score(encoded_train['feature_0'], df_outs_train['y_y']))
print(r2_score(encoded_train['feature_1'], df_outs_train['y_x']))

In [ ]:
def change_and_check(df_features_train_ori,feat,modifier):
    df_features_train = df_features_train_ori.copy()
    df_features_train[feat] = df_features_train[feat] + modifier
    encoded_train = pd.DataFrame(encoder.predict([df_encoder_train,df_features_train]))
    encoded_train = encoded_train.add_prefix('feature_')
    X_t = df_features_train
    contra = X_t['X0_x']*X_t['X1_x']+X_t['X2_x']*X_t['X3_x']+X_t['X1_x']*X_t['X3_x'] + x4_train*((X_t['X1_x'])-X_t['X2_x'])
    contra2 = X_t['X0_y']+X_t['X1_y']+X_t['X2_y']+X_t['X3_y']+X_t['X0_y']*X_t['X1_y'] + x4_train*((X_t['X1_y'])-X_t['X2_y'])
    if feat[-1]=='x':
        print('MAE C1: ' + str(mean_absolute_error(encoded_train['feature_1'], contra)))
        print('R2 C1: ' + str(r2_score(contra, encoded_train['feature_1'])))
    if feat[-1]=='y':
        print('MAE C2: ' + str(mean_absolute_error( encoded_train['feature_0'],contra2)))
        print('R2 C2: ' + str(r2_score(contra2, encoded_train['feature_0'])))

In [ ]:
change_and_check(df_features_train,'X0_x',1)
 

In [ ]:
change_and_check(df_features_train,'X1_y',+0.3)

In [ ]:
change_and_check(df_features_train,'X1_x',-0.3)

In [ ]:
change_and_check(df_features_train,'X1_x',+0.3)

ys_train.iloc[0:2]

In [ ]:
contra.head()

In [ ]:
mse = mean_absolute_error(contra, encoded_train['feature_3'])
mse

In [ ]:
mpe_sum = 0
for real, prediction in zip(contra, encoded_train['feature_3']):
    mpe_sum += ((real - prediction)/real)
mpe = mpe_sum/len(contra)
print(mpe)

In [ ]:
mse / np.mean(contra)

In [ ]:
mse / np.std(contra)

In [ ]:
np.median(contra)

In [ ]:
np.std(contra)

In [ ]:
df_encoder_train = X_train.merge(X2_train,left_index=True,right_index=True)
df_encoder_test = X_test.merge(X2_test,left_index=True,right_index=True)
df_outs_train = df_encoder_train[['error_x','error_y','y_y','y_x']]
df_outs_test = df_encoder_test[['error_x','error_y','y_y','y_x']]
df_features_train = df_encoder_train.drop(['error_x','error_y','y_y','y_x'], axis = 1)
df_features_test = df_encoder_test.drop(['error_x','error_y','y_y','y_x'], axis = 1)